In [ ]:
import numpy as np
from bluesky import traf, stack, settings
from bluesky.tools.aero import ft, nm, kts
from bluesky.tools.geo import qdrpos, latlondist

In [ ]:
TOP = 18000     # Top altitude of TRACON in feet
BOTTOM = 5000   # Bottom altitude of TRACON in feet

In [ ]:
def init_plugin():
    # Create plugin instance
    global approachbs
    approachbs = ApproachBS()
    
    # Configuration parameters
    config = {
        'plugin_name': 'APPROACHBS',
        'plugin_type': 'sim',
        'update_interval': 0.1,  # Update interval in seconds
        'update': approachbs.update,  # Update function
        'preupdate': approachbs.preupdate,  # Preupdate function
        'reset': approachbs.reset  # Reset function
    }
    
    return config


In [9]:
import bluesky as bs
print(bs.scr.__class__)

<class 'bluesky.simulation.screenio.ScreenIO'>


In [10]:
import inspect
print(inspect.getfile(bs.scr.__class__))

c:\myprojects\venv\Lib\site-packages\bluesky\simulation\screenio.py
